This file contains the implementation of ridge (L2) regression for mapping fMRI embeddings to CLIP and word2vec embedding

First setup baseline model (word2vec) (Not yet implemented)

In [ ]:


# semeval-2016-2017-task3-subtaskA-unannotated
model = api.load("glove-twitter-25")  # download the model and return as object ready for use
# model.most_similar("cat")

In [31]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from scipy import signal
from pathlib import Path
from functools import reduce
# from google.colab import drive
import pickle
from PIL import Image
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# drive.mount('/content/drive')
fMRI_folder = Path('doi_10.5061_dryad.gt413__v1')

In [3]:
NUM_SUBJS = 8
subjects_fmri = [] #stores all 8 subject fmri np arrays

# fMRI_folder = Path('./doi_10.5061_dryad.gt413__v1')
# assert fMRI_folder.exists(), f"Foldder: {fMRI_folder} does not exist."

for subj_id in range(2):
    print("Subject:",subj_id)
#     fmri_file_name = str(subj_id) + '_masked_2d.npy'
#     fmri = np.load(fMRI_folder / fmri_file_name)
    fmri_file_name = str(subj_id) + '_smooth_nifti_4d.nii'
    fmri = nib.load(fMRI_folder / fmri_file_name)
    fmri = np.array(fmri.dataobj)
    assert isinstance(fmri, np.ndarray), f"Imported fmri_scan for subject {subj_id} is not of type numpy.ndarray"
    assert(fmri.ndim) == 4, f"Imported fmri_scan for subject {subj_id} is not 4 dimensional"
    subjects_fmri.append(fmri)


Subject: 0
Subject: 1


In [4]:
np.shape(subjects_fmri) 

(2, 53, 60, 50, 1311)

In [5]:
np.shape(subjects_fmri[1]) 

(53, 60, 50, 1311)

In [6]:
feature_matrix = np.zeros((5176,195)) #stores the feature vectors as a row for each word
feature_names = [] #stores the names of all features in order
feature_types = {} #stores the types of features and all the names of the features for each type

features = sio.loadmat(fMRI_folder/'story_features.mat')
feature_count = 0
for feature_type in features['features'][0]:
    feature_types[feature_type[0][0]] = []
    if isinstance(feature_type[1][0], str):
        feature_types[feature_type[0][0]].append(feature_type[1][0])
        feature_names.append(feature_type[1][0])
    else:
        for feature in feature_type[1][0]:
            feature_types[feature_type[0][0]].append(feature[0])
            feature_names.append(feature[0])
    feature_matrix[:, feature_count:feature_count+feature_type[2].shape[1]] = feature_type[2] #adds the (5176xN) feature values to the feature matrix for the current feature group
    feature_count += feature_type[2].shape[1]

In [7]:
words_info = [] #stores tuples of (word, time, features) sorted by time appeared

mat_file = 'subject_1.mat' #only looks at the first subject file, somewhere it said all the timings were the same so this should be safe
mat_contents = sio.loadmat(fMRI_folder/mat_file)
# print(mat_content)
for count, row in enumerate(mat_contents['words'][0]):
    word_value = row[0][0][0][0]
    time = row[1][0][0]
    
    word_tuple = (word_value, time, feature_matrix[count,:])
    words_info.append(word_tuple)

In [80]:
chapter_nine_text = ""
for row in mat_contents['words'][0]:
    chapter_nine_text += row[0][0][0][0] + " "
# print(chapter_nine_text)

In [10]:
for count, subject in enumerate(subjects_fmri):
    print(count, np.shape(subject))

0 (53, 60, 50, 1311)
1 (53, 60, 50, 1311)


In [11]:
with open('fmri_indices', 'rb') as f:
    fmri_indices = pickle.load(f)

Harrison's implementation of Gaussian weighting

In [65]:
def hrf_alignment(num_words, words_info, NUM_SUBJS, subjects_fmri):

    subjects_samples = [[] for i in range(NUM_SUBJS)] #stores lists of all the samples for each subject
    window = signal.windows.gaussian(16, std=1) #gaussian window for the 4 fMRI scans
    num_words = num_words
    word_count = 0
    while word_count < len(words_info) - 24:
        #gets the 4 input words, and the 4 consecutive words while verifying they were read in sequence
        scan_words = []
        start_time = words_info[word_count][1]
        in_sequence = True #tracks if the words are in sequence or not
        for i in range(num_words):
            word_info = words_info[word_count + i]
            if word_info[1] != start_time + 0.5*i:
                #if some of the words are not in sequence, skip forward 1 word after innter loop
                in_sequence = False
            scan_words.append(word_info[0])
        if not in_sequence:
            word_count +=1
            continue
        fmri_time = start_time #fMRI is taken at first word read
        fmri_index = fmri_time//2 #since a scan happens every two seconds, the index is the time divided by 2
        if not np.issubdtype(fmri_index, np.integer):
            #if the first word is not aligned with the fmri scan (i.e. its not the first word in a TR)
            word_count += 1
            continue
        break_found = False
        for count, subject in enumerate(subjects_fmri):
#             print("Subject:",count)
            #adds tuple of (fmri_scan, four words)
            for i in range(num_words): #time delay of reading word after fMRI scan
                delay = 0.5*i #delay after first word
                for j in range(1,5): #get next 4 fMRI scans
                    fMRI_delay = 2*j - delay #delay until jth next fMRI scan
                    weight = window[int(2*fMRI_delay)-1] #gets the gaussian weighting (16 points instead of 8 to allow for .5)
                    try:
                        curr_fmri_idx = fmri_indices.index((start_time + math.floor(i/4) + 2*j)//2) #gets the index of the jth next fMRI scan
                    except:
#                         print("break found at index:",(start_time + math.floor(i/4) + 2*j)//2)
                        break_found = True
                        break
                    #adds weight fMRI scan
                    if j == 1:
                        word_scan = weight*subject[:,:,:,curr_fmri_idx]
                    else:
                        word_scan += weight*subject[:,:,:,curr_fmri_idx]
                if break_found:
                    break
                if i == 0:
                    summed_weighted_scans = word_scan
                else:
                    summed_weighted_scans += word_scan
            if break_found:
                break
            subjects_samples[count].append((summed_weighted_scans/num_words, scan_words))
        if break_found:
            word_count += 1
            continue 
#         print("Created sample:")
#         print("\tScan time:", str(start_time))
#         print("\tInput words:", str(scan_words))
        #if successful, skip forward to the next set of 4 words
        word_count += 4
    return subjects_samples

    print("Total number of samples:", str(len(subjects_samples[0])))

In [55]:
def get_fMRI_embedding(subjects_samples):
    scaler = StandardScaler()

    flattened_shape = reduce(np.multiply, subjects_samples[0][0].shape)
    X_matrix = np.zeros((len(subjects_samples), flattened_shape))

    for idx, sample in enumerate(subjects_samples):
        tmp = sample[0]
        # Reshape the voxels 
        tmp = tmp.ravel()
        X_matrix[idx,:] = tmp
        
    # Apply voxelwise standardization        
    X_matrix = scaler.fit_transform(X_matrix)
    
    return X_matrix

In [87]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Jai's Implementation of clip embedding

In [88]:
def get_clip_embedding(subject_sample, model,processor):
    word_list = []
    for i in range(len(subject_sample)):
        word_list.append(subject_sample[i][1])
    new_word_list = [' '.join(sublist) for sublist in word_list]
    
    inputs = processor(text=new_word_list, images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        # text_embeds is in the same space as the given image
        text_embeds = outputs['text_embeds']

        # pooler_output is just the text embedding 
        # This is what we want to use I think 
        pooler_output = outputs['text_model_output']['pooler_output']
#         print(pooler_output.numpy().shape)
    return pooler_output

In [67]:
# split data into train and test

def split_data(X_matrix, Y_matrix):
    X_train, X_test, y_train, y_test = train_test_split(X_matrix, y_matrix,
                                                        test_size=0.20,
                                                        shuffle=False)
    return X_train, X_test, y_train, y_test

In [68]:
# Train Ridge Regression Model

def train_model(X_train, y_train):
    
    ridge_model = Ridge(alpha=1.0)
    model = ridge_model.fit(X_train, y_train)
    
    return model


In [69]:
# Evaluate the model

def evaluate_model(model, X_test, y_test, random_text_embedding):
    correct_predictions = 0

    # Predict text embedding from test fMRI data
    y_pred = model.predict(X_test)
#     print(np.shape(y_pred))

    # Compute cosine similarity
    prediction_similarity_true = cosine_similarity(y_test, y_pred)
    prediction_similarity_random = cosine_similarity(random_text_embedding, y_pred)
#     print(np.shape(prediction_similarity_true), np.shape(prediction_similarity_true))
    
     # Count the occurrences of "1" in the first column    
    for i in range(X_test.shape[0]):
        if prediction_similarity_true[i].mean() < prediction_similarity_random[i].mean():
            correct_predictions +=1

    # Calculate the accuracy
    accuracy = (correct_predictions / X_test.shape[0])
    
    return accuracy


In [70]:
def obtain_two_way_class_result(model, X_test, y_test, y_matrix, batches):
    accuracy = []
    for i in range(batches):
        idx = np.arange(len(y_test))
        np.random.shuffle(idx)

        random_text_embedding = y_matrix[idx]

        test_accuracy = evaluate_model(model, X_test, y_test, random_text_embedding)
        accuracy.append(test_accuracy)

    avg_accuracy = sum(accuracy)/ batches
    
    print(accuracy)
    print(f"Average Accuracy: {avg_accuracy * 100:.2f}%")
    
    return avg_accuracy

In [89]:
context_lengths = [4,8,12,16]
NUM_SUBJS = 2
for length in context_lengths:
    # Step 1: Obtain aligned fMRI embedding with words
    print(f'Obtaining aligned fMRI scan for context length: {length}---------------------------------------------')
    subjects_samples = hrf_alignment(length, words_info, NUM_SUBJS, subjects_fmri)

    # Step 2: Obtain text and fMRI embedding
    # get embeddings for subject 1
    for subject in range(NUM_SUBJS):
        print(f'---------Obtaining clip and fMRI embedding for subject {subject}-----------------')
        y_matrix = get_clip_embedding(subjects_samples[subject], model, processor)
        X_matrix = get_fMRI_embedding(subjects_samples[subject])
        print(f'Subject {subject} \n fMRI Embedding Shape: {np.shape(X_matrix)} \n Text Embedding Shape: {np.shape(y_matrix)}')
        
        # Split data for training
        print(f'---------Spliting data for subject {subject}-----------------')
        X_train, X_test, y_train, y_test = split_data(X_matrix, y_matrix)
        
        print(f'---------Training started for subject {subject}-----------------')
        # Train model
        model_ = train_model(X_train, y_train)
        
        # Evaluate model
        print(f'---------Getting two way classification accuracy for subject {subject}-----------------\n')

        batches = 8
        accuracy = obtain_two_way_class_result(model_, X_test, y_test, y_matrix, batches)
        
    print(f'Subject: {subject} \n Context Length: {length} \n Average Accuracy: {accuracy}\n\n')



Obtaining aligned fMRI scan for context length: 4---------------------------------------------
---------Obtaining clip and fMRI embedding for subject 0-----------------
Subject 0 
 fMRI Embedding Shape: (1235, 159000) 
 Text Embedding Shape: torch.Size([1235, 768])
---------Spliting data for subject 0-----------------
---------Training started for subject 0-----------------
---------Getting two way classification accuracy for subject 0-----------------

[0.3724696356275304, 0.41295546558704455, 0.4008097165991903, 0.44129554655870445, 0.38866396761133604, 0.4048582995951417, 0.4089068825910931, 0.38461538461538464]
Average Accuracy: 40.18%
---------Obtaining clip and fMRI embedding for subject 1-----------------
Subject 1 
 fMRI Embedding Shape: (1235, 159000) 
 Text Embedding Shape: torch.Size([1235, 768])
---------Spliting data for subject 1-----------------
---------Training started for subject 1-----------------
---------Getting two way classification accuracy for subject 1--------